In [1]:
import pandas as pd 
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Data Load**

In [2]:
#####################################################################################
#The file name has to be replaced with the file name with is required for the present model
#with open(<<<ENTER YOUR FILE NAME HERE>>>, 'r', encoding="utf8") as file:
####################################################################################
train_df = pd.read_csv("train.data", 
                     sep="\t",
                     names=["Topic_Id","Topic_Names","Sent_1","Sent_2","Label","Sent_1_tag","Sent_2_tag"])
dev_df = pd.read_csv("dev.data", 
                     sep="\t",
                     names=["Topic_Id","Topic_Names","Sent_1","Sent_2","Label","Sent_1_tag","Sent_2_tag"])
test_df = pd.read_csv("test.data", 
                     sep="\t",
                     names=["Topic_Id","Topic_Names","Sent_1","Sent_2","Label","Sent_1_tag","Sent_2_tag"])

**Dropping Unused Columns**

In [3]:
dev_df = dev_df.drop(['Sent_1_tag', 'Sent_2_tag'], axis=1)
dev_df.head()


,Topic_Id,Topic_Names,Sent_1,Sent_2,Label
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,"(1, 4)"
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)"
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,"(1, 4)"
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)"
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,"(1, 4)"


In [4]:
dev_copy = dev_df.copy()

In [5]:
dev_df["Label"].value_counts()

(0, 5)    1748
(1, 4)     924
(2, 3)     585
(4, 1)     537
(3, 2)     522
(5, 0)     411
Name: Label, dtype: int64

**Binarize**

In [6]:
def transform_labels(dataset):
    label = []
    for index, row in dataset.iterrows():
        if row["Label"] in ["(3, 2)","(4, 1)","(5, 0)"]:
            label.append(True)
        else:
            label.append(False)    
    dataset["Label"] = label
    return dataset

In [7]:
dev_df = transform_labels(dataset=dev_df)

In [8]:
dev_df["Label"].value_counts()

False    3257
True     1470
Name: Label, dtype: int64

In [9]:
dev_df.head()

,Topic_Id,Topic_Names,Sent_1,Sent_2,Label
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,False
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,True
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,False
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,True
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,False


**Sentence List**

In [10]:
sentences1 = dev_df['Sent_1'].to_list()
sentences2 = dev_df['Sent_2'].to_list()

**TF-IDF  Baseline**

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def is_paraphrase(text_a, text_b):
    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the vectorizer on Text A and Text B
    tfidf_a = vectorizer.fit_transform([text_a])
    tfidf_b = vectorizer.transform([text_b])

    # Compute the cosine similarity between the TF-IDF vectors
    similarity = cosine_similarity(tfidf_a, tfidf_b)[0][0].astype(float)

    # Return "Yes" if the cosine similarity is above a threshold, otherwise "No"
    if similarity > 0.7:  # adjust the threshold value as needed
        return True  # They are paraphrases
    else:
        return False  # They are not paraphrases

# Define a list to store the results of the comparisons
baseline_results = []

# Loop over the sentence pairs and compute the similarity for each pair
for text_a, text_b in zip(sentences1, sentences2):
    result = is_paraphrase(text_a, text_b)
    baseline_results.append(result)

# Assign the list of results to the Baseline column
dev_df["Baseline"] = baseline_results


'''
for text_a, text_b in zip(sentences1, sentences2):
    result = is_paraphrase(text_a, text_b)
    dev_df["Baseline"] = result
'''

'\nfor text_a, text_b in zip(sentences1, sentences2):\n    result = is_paraphrase(text_a, text_b)\n    dev_df["Baseline"] = result\n'

In [12]:
dev_df.head(500)

,Topic_Id,Topic_Names,Sent_1,Sent_2,Label,Baseline
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,False,False
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,True,True
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,False,False
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,True,False
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,False,False
...,...,...,...,...,...,...
495,231,Calum,Calum I love you plz follow me,Can I please be in your timeline Calum,False,False
496,231,Calum,Calum I love you plz follow me,Really Calum please notice me,False,False
497,231,Calum,Calum I love you plz follow me,calum please see this and follow me I love u s...,True,True
498,231,Calum,Calum I love you plz follow me,please follow me CALUM im gonna cry,True,True


**Result TF-IDF baseline**

In [13]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

actual = dev_df["Label"]
predicted = dev_df["Baseline"]
predicted_b = predicted.apply(lambda x: True if x == 'True' else False)


scores_b = pd.DataFrame({
    'Precision': precision_score(actual, predicted_b),
    'Recall': recall_score(actual, predicted_b),
    'Accuracy': accuracy_score(actual, predicted_b),
    'F1-Score': f1_score(actual, predicted_b)
}, index = ['1. Cosine Similarity'])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=dbc0a0fd1a370a225034de92d6452ea1cb36437c7f9a71f71617572e47ca1dc2
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
print(classification_report(actual, predicted_b))

              precision    recall  f1-score   support

       False       0.69      1.00      0.82      3257
        True       0.00      0.00      0.00      1470

    accuracy                           0.69      4727
   macro avg       0.34      0.50      0.41      4727
weighted avg       0.47      0.69      0.56      4727



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**BERT - MPNET**

In [15]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

output_1 = []
#Output the pairs with their score
for i in range(len(sentences1)):
    output_1.append(("{:.4f}".format((cosine_scores[i][i]))))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
# Convert output_2 to a list of floats
my_float_list_1 = [float(x) for x in output_1]

# Add cosine similarities and algorithm columns to dataframe
dev_df["cosine_similarities_1"] = my_float_list_1
dev_df["cosine_similarities_1"] = dev_df["cosine_similarities_1"].astype(float) # ensure column is of float type
dev_df["Algo_1"] = dev_df["cosine_similarities_1"] > 0.7

In [17]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

actual = dev_df["Label"]
predicted = dev_df["Algo_1"]


scores = pd.DataFrame({
    'Precision': precision_score(actual, predicted),
    'Recall': recall_score(actual, predicted),
    'Accuracy': accuracy_score(actual, predicted),
    'F1-Score': f1_score(actual, predicted)
}, index = ['1. Cosine Similarity'])

In [18]:
dev_df[100:]


,Topic_Id,Topic_Names,Sent_1,Sent_2,Label,Baseline,cosine_similarities_1,Algo_1
100,72,Amber,These Amber Alerts need to chill,That s 5 amber alerts today Ive got,True,False,0.7169,True
101,72,Amber,These Amber Alerts need to chill,That s the third amber alert I got today,False,False,0.7485,True
102,72,Amber,These Amber Alerts need to chill,These amber alert messages are scary as fudge man,False,False,0.7537,True
103,72,Amber,These Amber Alerts need to chill,Well that Amber Alert alert just scared tf out...,False,False,0.7044,True
104,72,Amber,My phone is annoying me with these amber alerts,All these amber alerts leave people s kids alone,False,False,0.6124,False
...,...,...,...,...,...,...,...,...
4722,1902,iTunes,My first podcast drops on iTunes,No wonder I spend so much on iTunes,False,False,0.3873,False
4723,1902,iTunes,My first podcast drops on iTunes,Our Dead and Gone EP is now on iTunes,False,False,0.4920,False
4724,1902,iTunes,My first podcast drops on iTunes,amslingshots is 7 on the iTunes charts,False,False,0.2386,False
4725,1902,iTunes,My first podcast drops on iTunes,iTunes deleted my voice memos,False,False,0.3161,False


**BERT-MPNET Result**

In [19]:
print(classification_report(actual, predicted))

              precision    recall  f1-score   support

       False       0.81      0.91      0.86      3257
        True       0.73      0.52      0.61      1470

    accuracy                           0.79      4727
   macro avg       0.77      0.72      0.73      4727
weighted avg       0.79      0.79      0.78      4727



**SIM-CSE**

In [20]:
!pip install transformers
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

#Calculate Similarity
output_2 = []
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")


for i in range (len(sentences1)):# Tokenize input texts
  texts = [sentences1[i],sentences2[i]]
  inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
  with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

# Calculate cosine similarities
# Cosine similarities are in [-1, 1]. Higher means more similar
  cosine_sim_0_1 = 1 - cosine(embeddings[0], embeddings[1])
  output_2.append(cosine_sim_0_1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# Convert output_2 to a list of floats
my_float_list_2 = [float(x) for x in output_2]

# Add cosine similarities and algorithm columns to dataframe
dev_df["cosine_similarities_2"] = my_float_list_2
dev_df["cosine_similarities_2"] = dev_df["cosine_similarities_2"].astype(float) # ensure column is of float type
dev_df["Algo_2"] = dev_df["cosine_similarities_2"] > 0.7

In [27]:
dev_df[200:400]

,Topic_Id,Topic_Names,Sent_1,Sent_2,Label,Baseline,cosine_similarities_1,Algo_1,cosine_similarities_2,Algo_2
200,122,Backstrom,Backstrom hurt himself during warmups,Also hate that Backstrom went down in the warm up,True,False,0.7352,True,0.617412,False
201,122,Backstrom,Backstrom hurt himself during warmups,Backstrom is out we lose,False,False,0.6566,False,0.377604,False
202,122,Backstrom,Backstrom hurt himself during warmups,Backstrom just went down during warmups,True,True,0.9018,True,0.744621,True
203,122,Backstrom,Backstrom hurt himself during warmups,Hopefully Backstrom is back soon,False,False,0.7359,True,0.379438,False
204,122,Backstrom,Backstrom hurt himself during warmups,Man if Backstrom is out that s huuuuge,False,False,0.6745,False,0.430496,False
...,...,...,...,...,...,...,...,...,...,...
395,162,Blake Griffin,Marc Gasol is having his way against Blake Gri...,I hate Chris Paul and Blake Griffin,False,False,0.4715,False,0.279929,False
396,162,Blake Griffin,Marc Gasol is having his way against Blake Gri...,The only thing blake griffin knows how to do i...,False,False,0.5198,False,0.318171,False
397,162,Blake Griffin,Marc Gasol is having his way against Blake Gri...,What did Blake Griffin do on that foul,False,False,0.5245,False,0.493499,False
398,162,Blake Griffin,Marc Gasol is having his way against Blake Gri...,Who plays the young Blake Griffin in the Kia c...,False,False,0.2816,False,0.455270,False


In [23]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

actual = dev_df["Label"]
predicted_2 = dev_df["Algo_2"]


scores_2 = pd.DataFrame({
    'Precision': precision_score(actual, predicted_2),
    'Recall': recall_score(actual, predicted_2),
    'Accuracy': accuracy_score(actual, predicted_2),
    'F1-Score': f1_score(actual, predicted_2)
}, index = ['1. Cosine Similarity'])

**SIM-CSE Result**

In [24]:
print(classification_report(actual, predicted_2))

              precision    recall  f1-score   support

       False       0.75      0.94      0.84      3257
        True       0.72      0.31      0.43      1470

    accuracy                           0.75      4727
   macro avg       0.73      0.63      0.64      4727
weighted avg       0.74      0.75      0.71      4727

